In [1]:
import pandas as pd
import numpy as np
from timedelta import Timedelta 

from lumibot.brokers import Alpaca
from lumibot.backtesting import BacktestingBroker, YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from lumibot.entities import Asset
from datetime import datetime 
from alpaca_trade_api import REST 

from src.get_data import GetUpdateData
from src.get_data import GetDataFRED
from ta import add_all_ta_features

from joblib import dump, load

import src.ml_algo_trade
from imp import reload
reload(src.ml_algo_trade)

C:\Users\18327\AppData\Local\Temp\ipykernel_11032\451146756.py:20: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


<module 'src.ml_algo_trade' from 'c:\\Users\\18327\\Desktop\\Python Projects\\stock_prediction\\src\\ml_algo_trade.py'>

In [2]:
import json

# Load secrets from secrets.json
with open('secrets.json') as f:
    secrets = json.load(f)
    
api_key_twelve = secrets['default']['twelve_data_api']
api_key_fred = secrets['default']['fred_api']

api_key_alpaca = secrets['default']['alpaca_api_key']
api_secret_alpaca = secrets['default']['alpaca_api_secret']

In [3]:
clf_model = load("models/candidate_models/lgbm-direction-classifier-2024-05-13-19-23-24.joblib")

In [8]:
clf_model.feature_names_in_

array(['open', 'high', 'low', 'close', 'volume', 'GICS Sector',
       'GICS Sub-Industry', 'volume_adi', 'volume_obv', 'volume_cmf',
       'volume_fi', 'volume_em', 'volume_sma_em', 'volume_vpt',
       'volume_vwap', 'volume_mfi', 'volume_nvi', 'volatility_bbm',
       'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch',
       'volatility_dcm', 'volatility_dcw', 'volatility_dcp',
       'volatility_atr', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg',
       'trend_vortex_ind_diff', 'trend_trix', 'trend_mass_index',
       'trend_dpo', 'trend_kst', 'trend_kst_sig', 'trend_kst_

# BackTesting

In [13]:
df_stock = (
    GetUpdateData(api_key=api_key_twelve)
    .get_data_single_ticker('HE')
    .sort_values("datetime", ascending=True)
)

_, df_tickers = GetUpdateData(
    update_existing=False, api_key=api_key_twelve
).get_tickers()

df_stock = df_stock.merge(
    df_tickers[["ticker", "GICS Sector", "GICS Sub-Industry"]],
    on="ticker",
    how="left",
).drop('ticker', axis=1)

df_fred = GetDataFRED(api_key=api_key_fred).get_data()
df_fred["datetime"] = pd.to_datetime(df_fred["datetime"])

df_ohlci = add_all_ta_features(
    df_stock,
    open="open",
    high="high",
    low="low",
    close="close",
    volume="volume",
    fillna=True,
)

df_ohlci["datetime"] = pd.to_datetime(df_ohlci["datetime"])
df_ohlci = df_ohlci.merge(df_fred, on="datetime", how="left")

df_ohlci["month"] = df_ohlci["datetime"].dt.month
df_ohlci["dayofweek"] = df_ohlci["datetime"].dt.weekday

Getting DGS10...
DGS10 [min-max] date: [1962-01-02 00:00:00-2024-05-10 00:00:00]
Getting FEDFUNDS...
FEDFUNDS [min-max] date: [1954-07-01 00:00:00-2024-04-01 00:00:00]
Getting GDP...
GDP [min-max] date: [1946-01-01 00:00:00-2024-01-01 00:00:00]
Getting UNRATE...
UNRATE [min-max] date: [1948-01-01 00:00:00-2024-04-01 00:00:00]
Getting CP...
CP [min-max] date: [1946-01-01 00:00:00-2023-10-01 00:00:00]
Getting WM2NS...
WM2NS [min-max] date: [1980-11-03 00:00:00-2024-04-01 00:00:00]
Getting NASDAQCOM...
NASDAQCOM [min-max] date: [1971-02-05 00:00:00-2024-05-10 00:00:00]
Getting UMCSENT...
UMCSENT [min-max] date: [1952-11-01 00:00:00-2024-03-01 00:00:00]
Getting HSN1F...
HSN1F [min-max] date: [1963-01-01 00:00:00-2024-03-01 00:00:00]
Getting MSPNHSUS...
MSPNHSUS [min-max] date: [1963-01-01 00:00:00-2024-03-01 00:00:00]


In [14]:
df_ohlci

,datetime,open,high,low,close,volume,GICS Sector,GICS Sub-Industry,volume_adi,volume_obv,...,FEDFUNDS,GDP,HSN1F,MSPNHSUS,NASDAQCOM,UMCSENT,UNRATE,WM2NS,month,dayofweek
0,2004-07-01,26.29,26.29,25.52,25.60,247500,NaN,NaN,-1.960714e+05,247500,...,24.75248,6.38592,-6.84932,11.67192,21.16612,6.38064,-11.29032,4.11772,7,3
1,2004-07-02,25.60,26.09,25.60,26.09,261400,NaN,NaN,6.532857e+04,508900,...,24.75248,6.38592,-6.84932,11.67192,21.16612,6.38064,-11.29032,4.11772,7,4
2,2004-07-06,26.09,26.10,25.91,25.91,275800,NaN,NaN,-2.104714e+05,233100,...,24.75248,6.38592,-6.84932,11.67192,12.42342,6.38064,-11.29032,3.73955,7,1
3,2004-07-07,25.88,26.00,25.83,25.97,145300,NaN,NaN,-1.164538e+05,378400,...,24.75248,6.38592,-6.84932,11.67192,12.51073,6.38064,-11.29032,3.73955,7,2
4,2004-07-08,25.82,26.00,25.76,25.85,149700,NaN,NaN,-1.538788e+05,228700,...,24.75248,6.38592,-6.84932,11.67192,12.79009,6.38064,-11.29032,3.73955,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-05-06,10.50,10.53,10.09,10.10,2102100,NaN,NaN,5.471387e+07,-58552407,...,10.35197,5.48564,8.28125,-1.86831,33.38791,28.06452,14.70588,0.83767,5,0
4996,2024-05-07,10.18,10.20,9.86,9.86,1747300,NaN,NaN,5.296657e+07,-60299707,...,10.35197,5.48564,8.28125,-1.86831,34.09822,28.06452,14.70588,0.83767,5,1
4997,2024-05-08,9.77,10.17,9.72,10.12,1924300,NaN,NaN,5.446325e+07,-58375407,...,10.35197,5.48564,8.28125,-1.86831,32.47340,28.06452,14.70588,0.83767,5,2
4998,2024-05-09,10.10,10.34,10.06,10.24,1910900,NaN,NaN,5.500922e+07,-56464507,...,10.35197,5.48564,8.28125,-1.86831,32.58910,28.06452,14.70588,0.83767,5,3


In [21]:
class MyStrategy(Strategy):
    def initialize(self, symbol: str = "SPY", cash_at_risk: float = 1):
        self.symbol = symbol
        self.budget = budget
        self.sleeptime = "7D"
        self.last_trade = None
        # self.set_market("24/7")
        self.cash_at_risk = cash_at_risk
        self.df_ohlci = self.get_data()

    def get_data(self):

        ticker = self.symbol.replace("-", "/")
        df_stock = (
            GetUpdateData(api_key=api_key_twelve)
            .get_data_single_ticker(ticker)
            .sort_values("datetime", ascending=True)
        )

        _, df_tickers = GetUpdateData(
            update_existing=False, api_key=api_key_twelve
        ).get_tickers()

        df_stock = df_stock.merge(
            df_tickers[["ticker", "GICS Sector", "GICS Sub-Industry"]],
            on="ticker",
            how="left",
        ).drop('ticker', axis=1)
        
        df_fred = GetDataFRED(api_key=api_key_fred).get_data()
        df_fred["datetime"] = pd.to_datetime(df_fred["datetime"])

        df_ohlci = add_all_ta_features(
            df_stock,
            open="open",
            high="high",
            low="low",
            close="close",
            volume="volume",
            fillna=True,
        )

        df_ohlci["datetime"] = pd.to_datetime(df_ohlci["datetime"])
        df_ohlci["month"] = df_ohlci["datetime"].dt.month
        df_ohlci["dayofweek"] = df_ohlci["datetime"].dt.weekday
        df_ohlci = df_ohlci.merge(df_fred, on="datetime", how="left")

        return df_ohlci

    def get_date(self):
        today = self.get_datetime()
        return today.strftime('%Y-%m-%d')

    def get_signal(self):
        
        # Need to use index lookup to overcome weekends with no data
        X = self.df_ohlci.copy()
        today = self.get_date()
        X = X.reset_index(drop=True)
        prev_index = X[X["datetime"] == today].index[0] - 1
        X = X.iloc[[prev_index]]
        
        X.drop('datetime', axis=1, inplace=True)

        pred_direction_proba = clf_model.predict_proba(X)[0][1]

        return pred_direction_proba

    def position_sizing(self):
        cash = self.cash
        last_price = self.get_last_price(self.symbol)
        quantity = cash * self.cash_at_risk // last_price
        return cash, last_price, quantity

    def on_trading_iteration(self):
        if broker.is_market_open():
            _, last_price, _ = self.position_sizing()
            print(f"Last Price: {last_price:.2f}")
            
            pred_direction_proba = self.get_signal()
            print(f"pred_direction_proba: {pred_direction_proba}")

            if (pred_direction_proba >= 0.5):
                if self.last_trade == "short":
                    self.sell_all()
                cash, last_price, quantity = self.position_sizing()
                
                if cash > last_price:
                    order = self.create_order(
                        self.symbol,
                        quantity,
                        "buy",
                        type="bracket",
                        take_profit_price=last_price * 1.10,
                        stop_loss_price=last_price * 0.95,
                    )
                    self.submit_order(order)
                    self.last_trade = "long"

            if (pred_direction_proba < 0.5): 
                if self.last_trade == "long":
                    self.sell_all()
                cash, last_price, quantity = self.position_sizing()
                
                if cash > last_price:
                    order = self.create_order(
                        self.symbol,
                        quantity,
                        "sell_short",
                        type="bracket",
                        take_profit_price=last_price * 0.90,
                        stop_loss_price=last_price * 1.05,
                    )
                    self.submit_order(order)
                    self.last_trade = "short"
        else:
            print("market is closed.")

# Backtesting

In [23]:
# Pick the dates that you want to start and end your backtest
# and the allocated budget
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 5, 12)
# broker = Alpaca(ALPACA_CREDS)

symbol = 'HSY'
budget = 1000

# Run the backtest
trader = Trader(backtest=True)
data_source = YahooDataBacktesting(
    datetime_start=start_date,
    datetime_end=end_date,
)
broker = BacktestingBroker(data_source)
strat = MyStrategy(
    symbol=symbol,
    broker=broker,
    budget=budget,
    benchmark_asset=symbol
)

trader.add_strategy(strat)
trader.run_all()

Getting DGS10...
DGS10 [min-max] date: [1962-01-02 00:00:00-2024-05-10 00:00:00]
Getting FEDFUNDS...
FEDFUNDS [min-max] date: [1954-07-01 00:00:00-2024-04-01 00:00:00]
Getting GDP...
GDP [min-max] date: [1946-01-01 00:00:00-2024-01-01 00:00:00]
Getting UNRATE...
UNRATE [min-max] date: [1948-01-01 00:00:00-2024-04-01 00:00:00]
Getting CP...
CP [min-max] date: [1946-01-01 00:00:00-2023-10-01 00:00:00]
Getting WM2NS...
WM2NS [min-max] date: [1980-11-03 00:00:00-2024-04-01 00:00:00]
Getting NASDAQCOM...
NASDAQCOM [min-max] date: [1971-02-05 00:00:00-2024-05-10 00:00:00]
Getting UMCSENT...
UMCSENT [min-max] date: [1952-11-01 00:00:00-2024-03-01 00:00:00]
Getting HSN1F...
HSN1F [min-max] date: [1963-01-01 00:00:00-2024-03-01 00:00:00]
Getting MSPNHSUS...
MSPNHSUS [min-max] date: [1963-01-01 00:00:00-2024-03-01 00:00:00]
Progress ||   0.09%  [Elapsed: 0:00:08 ETA: 2:40:19] Last Price: 147.23
pred_direction_proba: 0.5885530639082857
Progress ||   0.53%  [Elapsed: 0:00:09 ETA: 0:28:33] Portfoli

c:\Users\18327\Desktop\Python Projects\stock_prediction\.venv\lib\site-packages\lumibot\tools\indicators.py:56: RuntimeWarning:

invalid value encountered in scalar power

c:\Users\18327\Desktop\Python Projects\stock_prediction\.venv\lib\site-packages\lumibot\tools\indicators.py:56: RuntimeWarning:

invalid value encountered in scalar power

c:\Users\18327\Desktop\Python Projects\stock_prediction\.venv\lib\site-packages\lumibot\tools\indicators.py:56: RuntimeWarning:

invalid value encountered in scalar power




Creating trades plot...

Creating indicators plot...

Creating tearsheet...


{'MyStrategy': {'cagr': nan,
  'volatility': 3.9528710302007477,
  'sharpe': nan,
  'max_drawdown': {'drawdown': 1.015277924386236,
   'date': Timestamp('2021-10-05 10:30:00-0400', tz='America/New_York')},
  'romad': nan,
  'total_return': -1.8251306602478023}}

# Paper Trading

In [29]:
ALPACA_CONFIG = {
    "API_KEY": api_key_alpaca,
    "API_SECRET": api_secret_alpaca,
    "PAPER": True
}

symbol = 'SPY'

alpaca = Alpaca(ALPACA_CONFIG, connect_stream=False)
strategy = src.trading_bot.MLStrategy(
    symbol=symbol,
    clf_model=clf_model,
    reg_model=reg_model,
    broker=alpaca
)
trader = Trader()
trader.add_strategy(strategy)
trader.run_all()

2024-05-13 22:42:15,547: asyncio: INFO: [MLStrategy] Executing the initialize lifecycle method
2024-05-13 22:42:17,307: asyncio: INFO: [MLStrategy] Sleeping until the market opens
2024-05-13 22:42:17,905: asyncio: INFO: [MLStrategy] Executing the before_market_opens lifecycle method
2024-05-13 22:42:18,577: asyncio: INFO: [MLStrategy] Sleeping until the market opens
2024-05-13 22:42:18,578: asyncio: INFO: [MLStrategy] Executing the before_starting_trading lifecycle method
2024-05-13 22:42:19,453: asyncio: WARNING: [MLStrategy] The strategy will run at 09:30:05 every day. If instead you want to start right now and run every 7 days then set force_start_immediately=True in the strategy's initialization.
2024-05-13 22:42:19,487: asyncio: INFO: [MLStrategy] Strategy will check in again at: 2024-05-14 09:30:05
